## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

## Import Tables

In [2]:
# # Doug's Path
diagnoses_icd = pd.read_csv('./data/mimic-iv-0.4/hosp/diagnoses_icd.csv.gz', compression='gzip')
d_icd_diagnoses = pd.read_csv('./data/mimic-iv-0.4/hosp/d_icd_diagnoses.csv.gz', compression='gzip')
admissions = pd.read_csv('./data/mimic-iv-0.4/core/admissions.csv.gz', compression='gzip')
d_labitems = pd.read_csv('./data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

# Stephen's Path
# path = "D:\\Bootcamp\\MIMIC IV\\"
# admissions = pd.read_csv(path + "core\\admissions.csv.gz", compression='gzip')
# diagnoses_icd = pd.read_csv(path + "hosp\\diagnoses_icd.csv.gz", compression='gzip')
# d_icd_diagnoses = pd.read_csv(path + "hosp\\d_icd_diagnoses.csv.gz", compression='gzip')
# d_labitems = pd.read_csv(path + "hosp\\d_labitems.csv.gz", compression='gzip')

## Pull codes related to Congestive Heart Failure

In [3]:
# Congestive Heart Failure
chf_codes = list(map(str, d_icd_diagnoses[(d_icd_diagnoses['long_title'].str.lower().str.contains('congestive')) & (d_icd_diagnoses['long_title'].str.lower().str.contains('heart'))]['icd_code']))

# kidney_list = list(d_icd_diagnoses[(d_icd_diagnoses['long_title'].str.lower().str.contains('kidney')) & (d_icd_diagnoses['long_title'].str.lower().str.contains('failure'))]['icd_code'])
# pd.DataFrame(diagnoses_icd[diagnoses_icd['icd_code'].isin(kidney_list)].icd_code.value_counts()).reset_index().head(20).merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')

# ICD Codes we care about
# 5849, N179, 5845, N170
# akf_codes = ['5849   ', 'N179   ', '5845   ', 'N170   ']
chf_codes

['39891  ',
 '4280   ',
 'I5020  ',
 'I5021  ',
 'I5022  ',
 'I5023  ',
 'I5030  ',
 'I5031  ',
 'I5032  ',
 'I5033  ',
 'I5040  ',
 'I5041  ',
 'I5042  ',
 'I5043  ']

## Start filtering down to only records where AKF is True

In [5]:
diagnoses_icd_chf = diagnoses_icd[diagnoses_icd['icd_code'].isin(chf_codes)]


In [6]:
diagnoses_icd_chf.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
886,16569548,24870770,12,4280,9
1050,18460230,25771608,2,4280,9
2342,19661870,22135027,3,4280,9
2683,18932584,23973570,10,4280,9
2961,16723797,28913496,30,4280,9


## Generate new Admissions table with hospital stay duration and CHF flag

In [7]:
admissions_chf = admissions[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'ethnicity']]

admissions_chf['CHF_subj'] = np.where(admissions_chf.subject_id.isin(diagnoses_icd_chf.subject_id), 1, 0)
admissions_chf['CHF_adm'] = np.where(admissions_chf.hadm_id.isin(diagnoses_icd_chf.hadm_id), 1, 0)
admissions_chf['time_spent'] = pd.to_datetime(admissions_chf['dischtime']) - pd.to_datetime(admissions_chf['admittime'])


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to b

In [8]:
admissions_chf.head(10)

,subject_id,hadm_id,admittime,dischtime,ethnicity,CHF_subj,CHF_adm,time_spent
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,UNKNOWN,0,0,3 days 23:54:00
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,OTHER,0,0,2 days 14:45:00
2,14495017,22484010,2175-01-28 15:41:00,2175-01-29 16:00:00,WHITE,0,0,1 days 00:19:00
3,13676048,23865469,2193-01-19 05:27:00,2193-01-24 18:59:00,WHITE,0,0,5 days 13:32:00
4,13831972,27763544,2131-01-27 04:03:00,2131-01-27 05:39:00,WHITE,0,0,0 days 01:36:00
5,18523038,25414328,2142-08-26 17:14:00,2142-08-27 10:00:00,WHITE,0,0,0 days 16:46:00
6,16705931,20580522,2174-10-24 11:30:00,2174-10-24 18:45:00,WHITE,0,0,0 days 07:15:00
7,19963742,24951506,2171-07-04 15:58:00,2171-07-05 13:37:00,UNKNOWN,0,0,0 days 21:39:00
8,10903424,22568585,2181-01-31 13:09:00,2181-01-31 13:42:00,WHITE,0,0,0 days 00:33:00
9,15030422,21975876,2149-09-21 23:54:00,2149-09-22 03:59:00,WHITE,0,0,0 days 04:05:00


## Sample Target Patients for Modeling

In [9]:
target_sample = admissions_chf[['subject_id', 'CHF_subj']].drop_duplicates()

target_sample = resample(target_sample, n_samples = 10000, replace = False, stratify = target_sample.CHF_subj, random_state = 0)

print('Number of unique Patients: ', target_sample.subject_id.nunique())
print('\nSample Breakdown:')
print(target_sample.CHF_subj.value_counts())

Number of unique Patients:  10000

Sample Breakdown:
0    9179
1     821
Name: CHF_subj, dtype: int64


In [27]:
# identify first akf visit for NON-AKF patients
sample_non = target_sample[target_sample.CHF_subj==0]
admissions_non = admissions[admissions.subject_id.isin(sample_non.subject_id)]
admissions_non['admittime'] = pd.to_datetime(admissions_non.admittime)
first_non = admissions_non.loc[admissions_non.groupby('subject_id').admittime.idxmin()]
# sanity checks
print(first_non.shape)                 # number of rows
print(first_non.subject_id.nunique())  # matches number of patients
print(first_non.hadm_id.nunique())     # matches number of visits

# pull out hospital visits for target non-akf samples
sample_non = sample_non.merge(first_non, how='inner', on='subject_id')
sample_non = sample_non[['subject_id', 'hadm_id', 'CHF_subj']]
sample_non = sample_non.rename({'CHF_subj':'CHF'}, axis=1)

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(9179, 15)
9179
9179


In [28]:
# identify first chf visit for all CHF patients
sample_chf = target_sample[target_sample.CHF_subj==1]
first_chf = admissions_chf[admissions_chf.subject_id.isin(sample_chf.subject_id)]
first_chf['admittime'] = pd.to_datetime(first_chf.admittime)
first_chf = first_chf.loc[first_chf[first_chf.CHF_adm==1].groupby('subject_id').admittime.idxmin()]
first_chf = first_chf[['subject_id', 'hadm_id', 'CHF_adm']]
# sanity checks
print(np.mean(first_chf.CHF_adm))   # make sure everyone has AKF
print(first_chf.shape)                 # make sure number of rows...
print(first_chf.subject_id.nunique())  # ...matches number of patients
print(first_chf.hadm_id.nunique())     # ...matches number of visits

# pull out hospital visits for target akf samples
sample_chf = sample_chf.merge(first_chf, how='inner', on='subject_id')
sample_chf = sample_chf[['subject_id', 'hadm_id', 'CHF_adm']]
sample_chf = sample_chf.rename({'CHF_adm':'CHF'}, axis=1)

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


1.0
(821, 3)
821
821


In [29]:
# merge back together and write to file
admissions_sample = pd.concat([sample_chf, sample_non])
admissions_sample = admissions_sample.reset_index(drop=True)
print(admissions_sample.shape)                  # sanity check
print(admissions_sample.subject_id.nunique())   # sanity check

(10000, 3)
10000


Generate pickle file so we all work on the exact same data

In [30]:
admissions_sample.to_pickle('admissions_sample_chf.pkl')

In [13]:
# admissions_sample = pd.read_pickle("admissions_sample.pkl")

## Pull Chart and Lab data only where it matches our sample patient list

In [16]:
# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('./data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)


In [15]:
# chartevents_final.to_pickle("chartevents_final.pkl")

In [16]:
# chartevents_final = pd.read_pickle("chartevents_final.pkl")

In [17]:
chartevents_final.shape

(7602370, 10)

## Load Lab Events table

In [17]:
# %%time
chunksize = 3*(10 ** 6)
counter=0
lab_events_sample = []
for chunk in pd.read_csv('./data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
    lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
    counter+=1
    print(f'chunk {counter} processed')

lab_events_final = pd.concat(lab_events_sample)
# 3min 52s

chunk 1 processed
chunk 2 processed
chunk 3 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed


In [18]:
lab_events_final.to_pickle("lab_events_final_chf.pkl")

In [20]:
# lab_events_final = pd.read_pickle("lab_events_final.pkl")

In [19]:
lab_events_final.shape

(4573368, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [22]:
admissions_sample = pd.read_pickle("admissions_sample.pkl")

In [21]:
# lab_events_sampled = lab_events_final.copy()
%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(admissions_sample.hadm_id)]


CPU times: user 2.22 s, sys: 3.44 s, total: 5.66 s
Wall time: 11.7 s


Check how many patients have had each test done

In [22]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    6187
51265    6093
51222    6040
51301    6038
51248    6034
51249    6034
51277    6034
51279    6034
51250    6034
50971    5548
Name: subject_id, dtype: int64

### Most common tests performed on patients with CHF

In [31]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['CHF'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50971    799
50912    797
50983    797
51006    796
50902    796
50868    792
50882    792
51265    789
51221    789
50931    788
Name: subject_id, dtype: int64

In [33]:
# admissions_sample.sample(50)

Item Black List

In [40]:
item_black_list = [50920, 51466, 51486, 51487, 51464, 51484, 51478, 51514, 51492, 51506, 51491]
# 50920 - test results are text in comments - need to come back and figure out how to handle this
# 51466 - measures blood in urine; value and comments sometimes differ; need to add mapping
# 51486 - same as above, leukocytes in the urine
# 51487 - same as above, nitrite in the urine
# 51464 - same as above, bilirubin in the urine
# 51484 - same as above, ketone in the urine
# 51478 - same as above, glucose in the urine
# 51514 - same as above, urobilinogen in the urine
# 51492 - same as above, protein in the urine
# 51506 - same as above, urine appearance

Take only the X most commonly performed tests

In [35]:
top_test_num = 30

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with AKF
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['CHF'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]

lab_events_sampled_sub


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
23611,30566894,12469262,25365926.0,10015871,51237,2177-02-09 06:25:00,2177-02-09 07:18:00,1.8,1.80,NaN,0.9,1.1,abnormal,ROUTINE,NaN
23612,30566895,12469262,25365926.0,10015871,51274,2177-02-09 06:25:00,2177-02-09 07:18:00,19.2,19.20,sec,9.4,12.5,abnormal,ROUTINE,NaN
23613,30566896,12469262,25365926.0,39375115,51221,2177-02-09 06:25:00,2177-02-09 07:09:00,23.9,23.90,%,36.0,48.0,abnormal,ROUTINE,NaN
23614,30566897,12469262,25365926.0,39375115,51222,2177-02-09 06:25:00,2177-02-09 07:09:00,7.7,7.70,g/dL,12.0,16.0,abnormal,ROUTINE,NaN
23615,30566898,12469262,25365926.0,39375115,51248,2177-02-09 06:25:00,2177-02-09 07:09:00,37.4,37.40,pg,27.0,32.0,abnormal,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122235567,29776052,12408953,21986775.0,92842480,51250,2156-08-18 06:31:00,2156-08-18 07:27:00,93,93.00,fL,82.0,98.0,NaN,ROUTINE,NaN
122235568,29776053,12408953,21986775.0,92842480,51265,2156-08-18 06:31:00,2156-08-18 07:27:00,403,403.00,K/uL,150.0,440.0,NaN,ROUTINE,NaN
122235569,29776054,12408953,21986775.0,92842480,51277,2156-08-18 06:31:00,2156-08-18 07:27:00,17.4,17.40,%,10.5,15.5,abnormal,ROUTINE,NaN
122235570,29776055,12408953,21986775.0,92842480,51279,2156-08-18 06:31:00,2156-08-18 07:27:00,3.09,3.09,m/uL,4.2,5.4,abnormal,ROUTINE,NaN


In [36]:
lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 50920].comments.value_counts()

Series([], Name: comments, dtype: int64)

## Code to look into specific test values

In [29]:
# lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).count()


In [30]:
lab_events_sampled_sub_tiny = lab_events_sampled_sub[(lab_events_sampled_sub['subject_id'] == 19988493) & (lab_events_sampled_sub['hadm_id'] == 25600709.0) & (lab_events_sampled_sub['itemid'].isin([51237]))]#.valuenum.describe()
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['subject_id'] == 19988493) & (lab_events_sampled_sub['hadm_id'] == 25600709.0) & (lab_events_sampled_sub['itemid'].isin([51478, 51484])),]



In [31]:
# lab_events_sampled_sub_tiny['comments'] = lab_events_sampled_sub_tiny['comments'].replace(np.nan, 'NA')

lab_events_sampled_sub_tiny


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
116765206,124205250,19988493,25600709.0,29865343,51237,2116-09-26 23:36:00,2116-09-27 00:26:00,1.1,1.1,NaN,0.9,1.1,NaN,STAT,NaN
116765231,124205276,19988493,25600709.0,1870784,51237,2116-09-27 13:35:00,2116-09-27 14:09:00,1.2,1.2,NaN,0.9,1.1,abnormal,STAT,NaN
116765292,124205338,19988493,25600709.0,57401788,51237,2116-09-28 03:53:00,2116-09-28 05:02:00,1.0,1.0,NaN,0.9,1.1,NaN,ROUTINE,NaN
116765394,124205451,19988493,25600709.0,75955093,51237,2116-09-30 06:20:00,2116-09-30 08:16:00,1.0,1.0,NaN,0.9,1.1,NaN,ROUTINE,NaN


## Special updates for Urine sample tests

In [32]:
print(lab_events_sampled_sub[(lab_events_sampled_sub['itemid'] == 51484)].value.unique())

print(lab_events_sampled_sub[(lab_events_sampled_sub['itemid'] == 51478)].value.unique())


[]
[]


In [33]:
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8', '>19.2', 'ERROR', 'UNABLE TO REPORT'])), ]


In [37]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [38]:
# Used to investigate bad data

lab_events_sampled_sub[lab_events_sampled_sub['value'] == ' ']


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
26820103,123256387,19913465,28293201.0,90246682,51491,2167-08-04 17:00:00,2167-08-04 17:51:00,,NaN,units,5.0,8.0,NaN,STAT,UNABLE TO REPORT.
119090733,1874983,10153439,22115349.0,60839603,51491,2121-03-14 21:17:00,2121-03-15 07:46:00,,NaN,units,5.0,8.0,NaN,ROUTINE,UNABLE TO REPORT.


In [36]:
# Used to investigate bad data
pd.set_option('display.max_rows', 60)

lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 51498].value.value_counts().reset_index().sort_values(by = 'index')


,index,value
44,,3
56,1.000,1
45,1.001,3
38,1.002,6
33,1.003,9
23,1.004,30
19,1.005,38
16,1.006,53
15,1.007,59
7,1.008,79


In [37]:
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466)]['value'].map({'' : 0.0, 'NEG' : 0.0, 'TR' : 1.0, 'SM' : 2.0, 'MOD' : 3.0, 'LG' : 4.0, 'LGE' : 4.0}).fillna(0.0)
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466)]['value']
# = lab_events_sampled_sub['GarageFinish'].map({'NA' : 0.0, 'Unf' : 1.0, 'RFn' : 2.0, 'Fin' : 3.0})


In [44]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [39]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)), ]


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
336856,33996144,12745561,20731073.0,57802208,51498,2181-01-08 20:52:00,2181-01-08 20:59:00,1.007,1.007,,1.001,1.035,NaN,STAT,NaN
1409444,45059123,13629545,25775076.0,90762997,51498,2165-01-26 10:29:00,2165-01-26 11:31:00,1.017,1.017,,1.001,1.035,NaN,ROUTINE,NaN
1666394,47199633,13809869,27450240.0,16343571,51498,2118-05-14 23:23:00,2118-05-15 00:52:00,1.007,1.007,,1.001,1.035,NaN,ROUTINE,NaN
2273592,53256330,14285139,26961757.0,44892615,51498,2123-09-06 10:52:00,2123-09-06 11:39:00,1.007,1.007,,1.001,1.035,NaN,ROUTINE,NaN
2381428,54601194,14394997,20327732.0,62837261,51498,2157-12-31 23:05:00,2157-12-31 23:27:00,1.017,1.017,,1.001,1.035,NaN,STAT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120434256,11150480,10915996,22515797.0,34269915,51498,2178-09-21 14:16:00,2178-09-21 17:32:00,1.007,1.007,,1.001,1.035,NaN,ROUTINE,NaN
120436628,11178342,10918410,21954505.0,65846117,51498,2148-01-17 17:00:00,2148-01-17 17:25:00,1.027,1.027,,1.001,1.035,NaN,ROUTINE,NaN
120951761,16272155,11319328,29695432.0,17340353,51498,2142-06-18 22:06:00,2142-06-18 22:43:00,1.017,1.017,,1.001,1.035,NaN,STAT,NaN
121517405,22034085,11791735,26645051.0,65960542,51498,2181-02-22 10:03:00,2181-02-22 10:46:00,1.019,1.019,,1.001,1.035,NaN,ROUTINE,NaN


In [40]:
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['subject_id'] == 19988493) & (lab_events_sampled_sub['hadm_id'] == 25600709.0) & (lab_events_sampled_sub['itemid'].isin([51478, 51484])),]


## Get Aggregate Test Values

In [45]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


CPU times: user 503 ms, sys: 191 ms, total: 694 ms
Wall time: 1.25 s


In [46]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [47]:
# lab_events_sampled_sub_grouped[lab_events_sampled_sub_grouped['itemid'] == 51221]

missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

min     1234
max     1234
mean    1234
dtype: int64

In [48]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


In [49]:
lab_events_sampled_sub_grouped

,subject_id,hadm_id,itemid,count,min,max,mean,abn_percent,range_min,range_max,below_min,above_max
0,10002264,23205369.0,50885,1,4.10,4.10,4.100000,0.000000,0.0,12.0,0.00,0.0
1,10002495,24982426.0,50861,3,36.00,44.00,39.333333,0.333333,0.0,40.0,0.00,4.0
2,10002495,24982426.0,50863,3,75.00,99.00,86.000000,0.000000,40.0,130.0,0.00,0.0
3,10002495,24982426.0,50868,10,15.00,23.00,18.900000,0.200000,8.0,20.0,0.00,3.0
4,10002495,24982426.0,50878,3,32.00,210.00,96.333333,0.666667,0.0,40.0,0.00,170.0
...,...,...,...,...,...,...,...,...,...,...,...,...
140777,19999204,29046609.0,51275,3,31.80,34.70,33.200000,0.000000,25.0,36.5,0.00,0.0
140778,19999204,29046609.0,51277,8,15.80,16.10,15.925000,1.000000,10.5,15.5,0.00,0.6
140779,19999204,29046609.0,51279,8,3.34,3.56,3.472500,1.000000,4.6,6.1,1.26,0.0
140780,19999204,29046609.0,51301,8,7.80,11.70,9.325000,0.250000,4.0,10.0,0.00,1.7


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [46]:
# Here is where we merge Admissions

# lab_events_sampled_sub_grouped.merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')


In [50]:
# For now aggregate all patients together

lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max.reset_index(inplace = True)

lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
lab_range_dic


{50861: (0.0, 40.0),
 50863: (37.366042345276874, 125.65757328990227),
 50868: (8.355709595038308, 19.572053994892375),
 50878: (0.0, 40.0),
 50882: (22.0, 32.0),
 50885: (0.0, 4.034812141795797),
 50893: (8.406765327696299, 10.290761099366144),
 50902: (96.0, 108.0),
 50911: (0.0, 10.0),
 50912: (0.44450771491552543, 1.1445077149154852),
 50931: (69.86519840516424, 100.84203531422062),
 50960: (1.598434891485942, 2.593739565943461),
 50970: (2.7223794212220396, 4.536227224008577),
 50971: (3.328334534967258, 5.142501802450993),
 50983: (133.35400797968808, 145.35400797968808),
 51003: (0.0, 0.009999999999999912),
 51006: (6.0, 20.0),
 51221: (37.33020850169711, 49.35930176175852),
 51222: (12.559139072847234, 16.73243377483429),
 51237: (0.9000000000000571, 1.0999999999999366),
 51248: (26.327974809413323, 31.912993039443155),
 51249: (31.322671528007955, 35.64534305601591),
 51250: (81.65197215777262, 98.34802784222738),
 51265: (150.0, 425.67536517314954),
 51274: (9.747572815534639

Pivot the table so we have feature columns related to test results

In [48]:
lab_events_sampled_sub_grouped[lab_events_sampled_sub_grouped['hadm_id'] == 23248240.0]

,subject_id,hadm_id,itemid,count,min,max,mean,abn_percent,range_min,range_max,below_min,above_max


In [51]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [52]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


CPU times: user 118 ms, sys: 30.4 ms, total: 148 ms
Wall time: 359 ms


In [53]:
pd.options.display.max_columns = 500

lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [52]:
# missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
# missing_vals[missing_vals > 0].sort_values()

## Start Data Imputation

Copy table, retain original for Random Forest

In [54]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [54]:
# lab_events_sampled_pivot = lab_events_impute

In [55]:
# d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')


In [55]:
# lab_events_impute.columns

d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

,itemid,label,fluid,category,loinc_code
52,50861,Alanine Aminotransferase (ALT),Blood,Chemistry,1742-6
76,50863,Alkaline Phosphatase,Blood,Chemistry,6768-6
115,50868,Anion Gap,Blood,Chemistry,1863-0
153,50878,Asparate Aminotransferase (AST),Blood,Chemistry,1920-8
212,50882,Bicarbonate,Blood,Chemistry,1963-8
229,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
282,50893,"Calcium, Total",Blood,Chemistry,2000-8
442,50902,Chloride,Blood,Chemistry,2075-0
511,50911,"Creatine Kinase, MB Isoenzyme",Blood,Chemistry,6773-6
512,50912,Creatinine,Blood,Chemistry,2160-0


In [56]:
%%time

np.random.seed(0)

for labitem in lab_range_dic:
    for ind in lab_events_sampled_pivot[lab_events_sampled_pivot[('mean', labitem)].isnull()].index:
        val_max = lab_range_dic[labitem][1]
        val_min = lab_range_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_sampled_pivot[('min', labitem)][ind]
        val_max = lab_events_sampled_pivot[('max', labitem)][ind]
        val_mean = lab_events_sampled_pivot[('mean', labitem)][ind]
        
        lab_events_sampled_pivot[('min', labitem)][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_sampled_pivot[('max', labitem)][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_sampled_pivot[('mean', labitem)][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 3min 2s, sys: 4.65 s, total: 3min 7s
Wall time: 6min 24s


In [59]:
for lab_itemid in lab_range_dic:
    lab_events_sampled_pivot[( 'above_max', lab_itemid)] = lab_events_sampled_pivot[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_sampled_pivot[( 'below_min', lab_itemid)] = lab_events_sampled_pivot[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_sampled_pivot[( 'abn_percent', lab_itemid)] = lab_events_sampled_pivot[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [60]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [63]:
lab_events_sampled_pivot.isna().sum()

            itemid
min         50861     0
            50863     0
            50868     0
            50878     0
            50882     0
                     ..
above_max   51279     0
            51301     0
            51491     0
subject_id            0
hadm_id               0
Length: 182, dtype: int64

In [65]:
lab_events_impute_KNN = lab_events_impute.copy()

In [66]:
lab_events_impute_KNN

min                                                                 \
itemid 50861  50863 50868 50878 50882 50885 50893  50902 50911 50912  50931   
0        NaN    NaN   NaN   NaN   NaN   4.1   NaN    NaN   NaN   NaN    NaN   
1       36.0   75.0  15.0  32.0  14.0   0.4   7.7   95.0  44.0   1.1  165.0   
2        NaN    NaN   9.0   NaN  23.0   NaN   NaN  103.0   NaN   1.0   96.0   
3        NaN    NaN   NaN   NaN   NaN   2.6   NaN    NaN   NaN   NaN    NaN   
4        NaN    NaN   NaN   NaN   NaN   7.6   NaN    NaN   NaN   NaN    NaN   
...      ...    ...   ...   ...   ...   ...   ...    ...   ...   ...    ...   
8162     NaN    NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN    NaN   
8163     NaN    NaN  12.0   NaN  26.0   NaN   9.1  104.0   NaN   1.5   89.0   
8164     9.0   64.0  10.0  11.0  16.0   0.3   7.5   86.0   NaN   0.8  148.0   
8165    15.0   57.0   5.0  25.0  22.0   0.8   7.8   92.0   NaN   0.5   76.0   
8166    28.0  112.0   7.0  51.0  27.0   1.0   8.1   90.0   NaN   0.8   81.0   

                                                                           \
itemid 50960 50970 50971  50983 51003 51006 51221 51222 51237 51248 51249   
0        NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1        1.7   3.0   3.5  129.0  0.69  30.0  33.1  11.1   1.1  30.4  32.1   
2        1.9   NaN   4.2  139.0   NaN  11.0  26.4   9.9   1.3  29.6  35.9   
3        NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4        NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...      ...   ...   ...    ...   ...   ...   ...   ...   ...   ...   ...   
8162     NaN   NaN   NaN    NaN   NaN   NaN  33.4  12.2   NaN  30.3  33.6   
8163     2.1   3.8   5.1  142.0   NaN  21.0  36.8  11.9   NaN  32.2  32.3   
8164     1.4   3.1   4.1  114.0   NaN  13.0  23.1   7.7   1.2  26.7  33.3   
8165     1.8   1.3   2.8  131.0   NaN  14.0  26.7   8.4   1.3  29.3  30.9   
8166     1.5   3.2   4.0  130.0   NaN  10.0  30.0   9.9   1.2  29.6  32.9   

                                                            max               \
itemid  51250  51265 51274 51275 51277 51279 51301 51491  50861  50863 50868   
0         NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
1        92.0  112.0  11.8  24.3  12.3  3.59  11.0   5.5   44.0   99.0  23.0   
2        80.0  151.0  14.6  34.8  13.9  3.28   7.0   NaN    NaN    NaN  13.0   
3         NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
4         NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
...       ...    ...   ...   ...   ...   ...   ...   ...    ...    ...   ...   
8162     90.0  205.0   NaN   NaN  13.8  4.03   8.5   NaN    NaN    NaN   NaN   
8163    100.0  201.0   NaN   NaN  12.7  3.69   8.9   NaN    NaN    NaN  12.0   
8164     77.0  226.0  13.2  29.2  11.7  2.85   7.9   6.0    9.0   64.0  18.0   
8165     90.0   81.0  14.4  25.9  16.8  2.80   5.4   6.5  118.0  149.0  16.0   
8166     88.0  124.0  13.2  31.8  15.8  3.34   7.8   6.0   41.0  122.0  11.0   

                                                                              \
itemid  50878 50882 50885 50893  50902  50911 50912  50931 50960 50970 50971   
0         NaN   NaN   4.1   NaN    NaN    NaN   NaN    NaN   NaN   NaN   NaN   
1       210.0  22.0   0.6   8.7  103.0  170.0   1.6  370.0   2.4   4.1   4.8   
2         NaN  31.0   NaN   NaN  113.0    NaN   1.6  138.0   2.3   NaN   4.6   
3         NaN   NaN   2.6   NaN    NaN    NaN   NaN    NaN   NaN   NaN   NaN   
4         NaN   NaN   7.6   NaN    NaN    NaN   NaN    NaN   NaN   NaN   NaN   
...       ...   ...   ...   ...    ...    ...   ...    ...   ...   ...   ...   
8162      NaN   NaN   NaN   NaN    NaN    NaN   NaN    NaN   NaN   NaN   NaN   
8163      NaN  26.0   NaN   9.1  104.0    NaN   1.5   89.0   2.1   3.8   5.1   
8164     11.0  23.0   0.3   8.7   97.0    NaN   1.2  312.0   2.6   4.0   5.4   
8165    180.0  33.0   3.0   9.5  120.0    NaN   0.8  225.0

In [67]:
missing_vals = lab_events_impute_KNN.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

             itemid
mean         51221     1980
below_min    51221     1980
abn_percent  51221     1980
above_max    51221     1980
max          51221     1980
                       ... 
min          50911     6912
mean         50911     6912
             51003     7460
max          51003     7460
min          51003     7460
Length: 180, dtype: int64

In [68]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

CPU times: user 2min 30s, sys: 28.5 s, total: 2min 59s
Wall time: 3min 22s


In [69]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns
lab_events_impute_KNN_imputed

min                                                                 \
itemid 50861  50863 50868 50878 50882 50885 50893  50902 50911 50912  50931   
0       30.6   88.6  13.3  32.1  22.6  4.10  8.53  101.8   3.1  0.93  103.9   
1       36.0   75.0  15.0  32.0  14.0  0.40  7.70   95.0  44.0  1.10  165.0   
2       98.1   87.2   9.0  46.6  23.0  1.36  8.61  103.0   3.5  1.00   96.0   
3       26.4   92.5  10.4  26.8  22.3  2.60  8.20   98.8  16.8  0.82   86.3   
4       28.1   86.4  11.0  36.8  23.5  7.60  8.17  102.2   3.1  0.90   92.2   
...      ...    ...   ...   ...   ...   ...   ...    ...   ...   ...    ...   
8162    45.1   95.4  11.0  43.2  22.9  1.07  8.42   99.0   5.0  0.93   92.7   
8163    26.9   96.3  12.0  35.3  26.0  1.37  9.10  104.0   3.5  1.50   89.0   
8164     9.0   64.0  10.0  11.0  16.0  0.30  7.50   86.0  14.3  0.80  148.0   
8165    15.0   57.0   5.0  25.0  22.0  0.80  7.80   92.0   3.9  0.50   76.0   
8166    28.0  112.0   7.0  51.0  27.0  1.00  8.10   90.0   7.1  0.80   81.0   

                                                                         \
itemid 50960 50970 50971  50983  51003 51006  51221  51222 51237  51248   
0       1.81  2.61  3.74  138.2  0.366  14.3  36.68  12.30  1.30  29.70   
1       1.70  3.00  3.50  129.0  0.690  30.0  33.10  11.10  1.10  30.40   
2       1.90  2.89  4.20  139.0  0.472  11.0  26.40   9.90  1.30  29.60   
3       1.63  2.62  3.45  133.8  0.362  11.2  35.33  12.39  1.07  32.81   
4       1.87  2.97  3.81  136.8  0.155  13.6  30.69  10.30  1.20  29.99   
...      ...   ...   ...    ...    ...   ...    ...    ...   ...    ...   
8162    1.92  3.17  3.84  135.6  0.429  19.6  33.40  12.20  1.57  30.30   
8163    2.10  3.80  5.10  142.0  0.153  21.0  36.80  11.90  1.12  32.20   
8164    1.40  3.10  4.10  114.0  0.798  13.0  23.10   7.70  1.20  26.70   
8165    1.80  1.30  2.80  131.0  0.237  14.0  26.70   8.40  1.30  29.30   
8166    1.50  3.20  4.00  130.0  0.372  10.0  30.00   9.90  1.20  29.60   

                                                                        max  \
itemid  51249  51250  51265  51274  51275  51277  51279  51301 51491  50861   
0       33.06   88.6  175.4  14.45  31.44  13.85  4.116   7.83  5.70   54.2   
1       32.10   92.0  112.0  11.80  24.30  12.30  3.590  11.00  5.50   44.0   
2       35.90   80.0  151.0  14.60  34.80  13.90  3.280   7.00  5.65  636.8   
3       32.94   97.8  200.5  11.96  27.13  15.39  3.717   9.97  5.85   69.7   
4       32.52   90.7  206.4  13.39  29.68  13.68  3.391   7.38  5.95   40.9   
...       ...    ...    ...    ...    ...    ...    ...    ...   ...    ...   
8162    33.60   90.0  205.0  16.52  28.82  13.80  4.030   8.50  6.00  214.0   
8163    32.30  100.0  201.0  12.49  28.17  12.70  3.690   8.90  5.95   41.6   
8164    33.30   77.0  226.0  13.20  29.20  11.70  2.850   7.90  6.00    9.0   
8165    30.90   90.0   81.0  14.40  25.90  16.80  2.800   5.40  6.50  118.0   
8166    32.90   88.0  124.0  13.20  31.80  15.80  3.340   7.80  6.00   41.0   

                                                                               \
itemid  50863 50868  50878 50882 50885 50893  50902  50911 50912  50931 50960   
0       104.9  15.9   42.6  25.7  4.10  8.99  105.9   18.6  1.07  156.3  2.08   
1        99.0  23.0  210.0  22.0  0.60  8.70  103.0  170.0  1.60  370.0  2.40   
2       113.8  13.0  462.3  31.0  2.32  9.05  113.0    6.4  1.60  138.0  2.30   
3       117.2  15.7  116.3  28.4  2.60  9.13  107.0   51.5  1.68  155.8  2.18   
4       131.7  14.1   61.4  27.3  7.60  8.84  107.4    3.7  1.20  148.2  2.18   
...       ...   ...    ...   ...   ...   ...    ...    ...   ...    ...   ...   
8162    107.9  15.6  181.3  28.1  1.54  8.89  104.3   11.2  1.11  179.5  2.33   
8163    120.3  12.0   54.4  26.0  1.57  9.10  104.0    5.3  1.50   89.0  2.10   
8164     64.0  18.0   11.0  23.0  0.30  8.70   97.0   41.1  1.20  312.0  2.60   
8165    149.0  16.0  180.0  33.0  3.00  9.50  120.0    5.4  0.80  225.0  2.80   
8166 

## Standardize the Data for use in Models

In [70]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [71]:
labs_scaled = lab_events_sampled_pivot
labs_scaled_KNN = lab_events_impute_KNN_imputed


In [72]:
labs_scaled

min                                                         \
itemid      50861       50863      50868      50878      50882     50885   
0        2.997088   53.363795  10.369141  -2.706156  23.468429  4.100000   
1       36.000000   75.000000  15.000000  32.000000  14.000000  0.400000   
2        8.502958   51.899458   9.000000   6.138574  23.000000  0.745549   
3       10.068224   48.020546   9.310767   3.595374  23.373409  2.600000   
4        5.192135   42.016499   9.261099   6.635144  24.476791  7.600000   
...           ...         ...        ...        ...        ...       ...   
8162     5.450012   60.737503   8.333361   6.911677  23.766951  1.047266   
8163     9.210654   42.542491  12.000000  -6.983840  26.000000  0.506439   
8164     9.000000   64.000000  10.000000  11.000000  16.000000  0.300000   
8165    15.000000   57.000000   5.000000  25.000000  22.000000  0.800000   
8166    28.000000  112.000000   7.000000  51.000000  27.000000  1.000000   

                                                                         \
itemid     50893       50902      50911     50912       50931     50960   
0       8.659014   96.556417   1.614418  0.510737   74.495080  1.828794   
1       7.700000   95.000000  44.000000  1.100000  165.000000  1.700000   
2       8.791798  103.000000   1.358330  1.000000   96.000000  1.900000   
3       8.511920   96.704762   2.099609  0.425821   77.253035  1.697534   
4       8.587909   97.204544   0.227240  0.536863   74.185054  1.799836   
...          ...         ...        ...       ...         ...       ...   
8162    8.523331   97.682902   1.729202  0.593280   71.259179  1.732312   
8163    9.100000  104.000000   0.650965  1.500000   89.000000  2.100000   
8164    7.500000   86.000000   2.382568  0.800000  148.000000  1.400000   
8165    7.800000   92.000000   0.772004  0.500000   76.000000  1.800000   
8166    8.100000   90.000000   0.938207  0.800000   81.000000  1.500000   

                                                                        \
itemid     50970     50971       50983     51003      51006      51221   
0       2.758343  3.544652  134.918019  0.001827   5.881766  39.617388   
1       3.000000  3.500000  129.000000  0.690000  30.000000  33.100000   
2       2.919012  4.200000  139.000000  0.002461  11.000000  26.400000   
3       3.124944  3.414196  133.659811  0.001490   8.466543  37.970242   
4       2.733779  3.675192  135.833220  0.001854   4.956332  39.241589   
...          ...       ...         ...       ...        ...        ...   
8162    2.848717  3.649594  136.699936  0.001313   7.346292  33.400000   
8163    3.800000  5.100000  142.000000  0.000235  21.000000  36.800000   
8164    3.100000  4.100000  114.000000  0.001202  13.000000  23.100000   
8165    1.300000  2.800000  131.000000  0.001252  14.000000  26.700000   
8166    3.200000  4.000000  130.000000  0.001793  10.000000  30.000000   

                                                                           \
itemid      51222     51237      51248      51249       51250       51265   
0       13.366440  0.899418  27.401276  31.615461   84.576850  212.953627   
1       11.100000  1.100000  30.400000  32.100000   92.000000  112.000000   
2        9.900000  1.300000  29.600000  35.900000   80.000000  151.000000   
3       13.205456  0.929930  27.397103  31.197589   81.857658  171.195602   
4       13.040595  0.899542  27.187877  32.371442   84.834987  186.776332   
...           ...       ...        ...        ...         ...         ...   
8162    12.200000  0.939502  30.300000  33.600000   90.000000  205.000000   
8163    11.900000  0.943021  32.200000  32.300000  100.000000  201.000000   
8164     7.700000  1.200000  26.700000  33.300000   77.000000  226.000000   
8165     8.400000  1.300000  29.300000  30.900000   90.000000   81.000000   
8166     9.900000  1.200000  29.600000  32.900000   88.000000  124.000000   

                                                                        \
itemid    

In [76]:
scaler.fit(labs_scaled.loc[:, ('min', 50861) : ('above_max', 51491)])
labs_scaled.loc[:, ('min', 50861) : ('above_max', 51491)] = scaler.transform(labs_scaled.loc[:, ('min', 50861) : ('above_max', 51491)])

scaler.fit(labs_scaled_KNN.loc[:, ('min', 50861) : ('above_max', 51491)])
labs_scaled_KNN.loc[:, ('min', 50861) : ('above_max', 51491)] = scaler.transform(labs_scaled_KNN.loc[:, ('min', 50861) : ('above_max', 51491)])


In [77]:
pd.DataFrame(labs_scaled_KNN)

min                                                              \
itemid     50861     50863     50868     50878     50882     50885     50893   
0      -0.186665 -0.193075  0.585371 -0.181148 -0.163053  0.401468  0.248314   
1      -0.128054 -0.417991  1.275651 -0.182491 -2.686965 -0.801902 -1.049778   
2       0.545964 -0.216228 -1.160633  0.013652 -0.045662 -0.489676  0.373431   
3      -0.232250 -0.128577 -0.592167 -0.252351 -0.251097 -0.086385 -0.267795   
4      -0.213799 -0.229458 -0.348539 -0.118006  0.101077  1.539790 -0.314714   
...          ...       ...       ...       ...       ...       ...       ...   
8162   -0.029285 -0.080617 -0.348539 -0.032025 -0.075010 -0.583994  0.076278   
8163   -0.226824 -0.065733  0.057509 -0.138158  0.834772 -0.486424  1.139774   
8164   -0.421106 -0.599909 -0.754586 -0.464616 -2.100009 -0.834425 -1.362571   
8165   -0.355983 -0.715675 -2.784823 -0.276533 -0.339140 -0.671808 -0.893381   
8166   -0.214884  0.193913 -1.972728  0.072764  1.128250 -0.606761 -0.424191   

                                                                              \
itemid     50902     50911     50912     50931     50960     50970     50971   
0       0.297256 -0.449481  0.003921  0.147048 -0.164456 -0.434913 -0.102946   
1      -1.256173  3.396581  0.275733  2.296938 -0.571876  0.088468 -0.661990   
2       0.571391 -0.411867  0.115844 -0.130925  0.168887 -0.059152  0.968556   
3      -0.388080  0.838809 -0.171957 -0.472234 -0.831143 -0.421493 -0.778457   
4       0.388634 -0.449481 -0.044045 -0.264634  0.057773  0.048208  0.060109   
...          ...       ...       ...       ...       ...       ...       ...   
8162   -0.342391 -0.270813  0.003921 -0.247040  0.242963  0.316609  0.129990   
8163    0.799836 -0.411867  0.915290 -0.377230  0.909650  1.162071  3.064972   
8164   -3.312183  0.603719 -0.203935  1.698769 -1.683021  0.222669  0.735621   
8165   -1.941510 -0.374253 -0.683602 -0.834654 -0.201494 -2.192937 -2.292536   
8166   -2.398401 -0.073338 -0.203935 -0.658722 -1.312639  0.356869  0.502686   

                                                                              \
itemid     50983     51003     51006     51221     51222     51237     51248   
0       0.432213 -0.125327 -0.084374  0.410294  0.397458  0.436923 -0.163384   
1      -2.003305  0.499912  1.648569 -0.093299 -0.087731 -0.370677  0.101788   
2       0.643998  0.079226 -0.448623 -1.035779 -0.572921  0.436923 -0.201265   
3      -0.732600 -0.133046 -0.426547  0.220392  0.433848 -0.491817  1.014736   
4       0.061591 -0.532504 -0.161639 -0.432311 -0.411191  0.033123 -0.053527   
...          ...       ...       ...       ...       ...       ...       ...   
8162   -0.256085 -0.003753  0.500632 -0.051099  0.357026  1.527183  0.063906   
8163    1.438189 -0.536363  0.655162  0.427175  0.235729 -0.289917  0.783657   
8164   -5.974260  0.708324 -0.227866 -1.499986 -1.462435  0.033123 -1.299833   
8165   -1.473845 -0.374264 -0.117488 -0.993579 -1.179407  0.436923 -0.314910   
8166   -1.738575 -0.113748 -0.559002 -0.529372 -0.572921  0.033123 -0.201265   

                                                                              \
itemid     51249     51250     51265     51274     51275     51277     51279   
0       0.198614 -0.280472 -0.433657  0.485161  0.234420 -0.250883  0.563089   
1      -0.484211  0.212417 -1.219678 -0.568664 -0.859517 -1.181776 -0.165073   
2       2.218639 -1.527191 -0.736163  0.544812  0.749214 -0.220854 -0.594219   
3       0.113261  1.053228 -0.122471 -0.505037 -0.425926  0.674004  0.010738   
4      -0.185475  0.023960 -0.049324  0.063631 -0.035234 -0.352981 -0.440557   
...          ...       ...       ...       ...       ...       ...       ...   
8162    0.582703 -0.077517 -0.066681  1.308338 -0.166997 -0.280912  0.444036   
8163   -0.341956  1.372156 -0.116273 -0.294272 -0.266585 -0.941545 -0.026639   
8164    0.369320 -1.962093  0.193673 -0.011926 -0.108776 -1.542121 -1.189485   
8165   -1.

## Check for Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
%%time

labs_scaled_features = labs_scaled.loc[:, ('min', 50868) : ('above_max', 51484)]
add_constant(labs_scaled_features)

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features.values, i) 
               for i in range(labs_scaled_features.shape[1])], 
              index=labs_scaled_features.columns).sort_values(ascending = False)

high_vif.head(20)


In [ ]:
labs_scaled_features_limited = labs_scaled_features.copy()
labs_scaled_features_limited.drop([('above_max', 51484),
                                  ('above_max', 51478),
                                  ('max', 51237),
                                  ('mean', 51248)], axis = 1, inplace = True)


In [ ]:
%%time

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features_limited.values, i) 
               for i in range(labs_scaled_features_limited.shape[1])], 
              index=labs_scaled_features_limited.columns).sort_values(ascending = False)

high_vif.head(20)


## Save Lab Results for Models

In [78]:
CHF_final_test = labs_scaled.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
CHF_KNN_final_test = labs_scaled_KNN.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [79]:
# admissions_sample.to_pickle("admissions_sample.pkl")
# admissions_sample.to_pickle("admissions_sample.pkl")
CHF_final_test.to_csv('CHF_final_test.csv.gz', compression="gzip")
CHF_KNN_final_test.to_csv('CHF_KNN_final_test.csv.gz', compression="gzip")
